<a href="https://colab.research.google.com/github/Eman-Adly/projects-NTI/blob/main/dental_classification_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dental Image Classification Model**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 📌 Step 1: Import Necessary Libraries
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout,Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report

## **Load Images and Labels**

In [3]:
# Function to load and resize images
def load_images_from_directory(directory, img_size=(224, 224)):
    images = []
    labels = []
    class_names = sorted(os.listdir(directory))  # Sorted for consistent labeling
    class_map = {class_name: idx for idx, class_name in enumerate(class_names)}  # Map class names to numbers

    for label, class_name in enumerate(class_names):
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):  # Ensure it's a directory
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_size[0], img_size[1]))
                img = tf.keras.preprocessing.image.img_to_array(img)
                images.append(img)
                labels.append(class_map[class_name])

    return np.array(images), np.array(labels), class_names

In [4]:
# Set dataset path
dataset_path = '/content/drive/MyDrive/Colab Notebooks/Cavity Dataset'

In [5]:
# Load images and labels
X, y, class_names = load_images_from_directory(dataset_path)

## **Images and Labels Splitting**

In [6]:
# Split into train and test (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Convert to NumPy arrays (for TensorFlow compatibility)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [8]:
x_test = X_test / 255.0

## **Images Preprocessing (Augmentation)**

In [9]:
#Define Constants
IMG_SIZE = 224  # Image size for MobileNetV2
BATCH_SIZE = 32  # Batch size for training
EPOCHS = 7  # Number of epochs before over fitting

In [10]:
# Data Preprocessing (Data Augmentation for Generalization)
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of training for validation
)

train_generator = train_datagen.flow(
    X_train,
    y_train,
    batch_size=BATCH_SIZE,
    subset='training'
)

val_generator = train_datagen.flow(
    X_train,
    y_train,
    batch_size=BATCH_SIZE,
    subset='validation'
)

In [11]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255) #rescal from [0:1] for beter learning
test_generator = test_datagen.flow(X_test, y_test, batch_size=32)

In [12]:
# 📌 Handle Class Imbalance (Compute Class Weights)
labels = y_train  # Get class labels from y_train instead of train_generator.classes
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
print("Class Weights:", class_weight_dict)

Class Weights: {0: np.float64(0.8671875), 1: np.float64(1.1808510638297873)}


## **Prepare the CNN Model**

In [13]:
# 📌 Load Pre-trained MobileNetV2 Model (Transfer Learning)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Freeze pre-trained layers

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [14]:
# 📌 Add Custom Layers for Dental Classification
x = base_model.output
x = GlobalAveragePooling2D()(x) #lower overfitting risk than maxpooling
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x) #to hinder overfitting
output_layer = Dense(1, activation='sigmoid')(x)  # Binary output (0 or 1)-->cavity or healthy

In [15]:
# 📌 Compile the Model
model = Model(inputs=base_model.input, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## **Fit the Model**

In [16]:
# 📌 Train the Model with Class Weights
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    class_weight=class_weight_dict
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/7
9/9 ━━━━━━━━━━━━━━━━━━━━ 29s 3s/step - accuracy: 0.6248 - loss: 0.7009 - val_accuracy: 0.7727 - val_loss: 0.6045
Epoch 2/7
9/9 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.8384 - loss: 0.3847 - val_accuracy: 0.8030 - val_loss: 0.4104
Epoch 3/7
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.8606 - loss: 0.3620 - val_accuracy: 0.8333 - val_loss: 0.4346
Epoch 4/7
9/9 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.8808 - loss: 0.2659 - val_accuracy: 0.8030 - val_loss: 0.3904
Epoch 5/7
9/9 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.8409 - loss: 0.3849 - val_accuracy: 0.8636 - val_loss: 0.3277
Epoch 6/7
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.8871 - loss: 0.2433 - val_accuracy: 0.8333 - val_loss: 0.3028
Epoch 7/7
9/9 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.9144 - loss: 0.1951 - val_accuracy: 0.8030 - val_loss: 0.3618


## **Model Evaluation**

In [17]:
# 📌 Evaluate the Model
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")


3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.8732 - loss: 0.3595
Validation Accuracy: 86.36%


In [18]:
# Predict using X_test
y_pred_probs = model.predict(x_test)  # Get predicted probabilities

# Convert probabilities to class labels (for binary classification)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()  # Convert to 0/1 labels

# Handle y_test correctly
if y_test.ndim > 1 and y_test.shape[1] == 2:  # If one-hot encoded
    y_true = np.argmax(y_test, axis=1)
else:  # Already in label format
    y_true = y_test.flatten()

# Define class labels manually (update based on your dataset)
class_labels = ['cavity', 'healthy']  # Replace with actual class names

# Print classification report
print(classification_report(y_true, y_pred, target_names=class_labels))


3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step
              precision    recall  f1-score   support

      cavity       0.90      0.90      0.90        48
     healthy       0.86      0.86      0.86        36

    accuracy                           0.88        84
   macro avg       0.88      0.88      0.88        84
weighted avg       0.88      0.88      0.88        84



## **Save the Model for Deployment**

In [19]:
# Save Model
model.save("dentical_model.h5")